Testing agent build in Langraph, integrating MCP server/tool for Brave Search

In [25]:
# Imports
from langchain_openai import ChatOpenAI

In [26]:
# Load env variables and instantiate LLM
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [27]:
from langchain_core.tools import tool
import requests
from typing import Optional, List, Dict, Any
from langchain.tools import tool

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

@tool
def brave_search(query : str, result_count : int = 1) -> List[Dict[str, Any]]:
    """
    Searches BraveSearch for the recent crypto information/news
    
    Args:
        query: The search query for crypto info (The market sentiment, recent legislation etc.)
        result_count: The number of results to return
        
    Returns:
        A list of news articles with titles, desc, URL, and published date
    """
    
    # Brave Search API Endpoint
    URL = "https://api.search.brave.com/res/v1/web/search"
    
     # Your Brave Search API key (store this securely in production)
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": "BRAVE_SEARCH_API_KEY"  # Replace with your actual API key
    }
    
    # Parameters for the API request
    params = {
        "q": f"{query} cryptocurrency market",
        "count": result_count,
        "freshness": "pd"  # Past day - for recent news
    }

    try:
        response = requests.get(URL, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        results = response.json()
    
        # format results
        formatted_results = []
        for item in results.get("news", []):
            formatted_results.append({
                "title": item.get("title"),
                "description": item.get("description"),
                "url": item.get("url"),
                "published_time": item.get("published_time"),
                "source": item.get("source")
            })
        
        return formatted_results
    
    except requests.RequestException as e:
        return [{"error": f"API request failed: {str(e)}"}]
    except (KeyError, ValueError) as e:
        return [{"error": f"Error parsing results: {str(e)}"}]
    
tools = [brave_search]

In [28]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

# Pass in:
# (1) the augmented LLM with tools
# (2) the tools list (which is used to create the tool node)
pre_built_agent = create_react_agent(llm, tools=tools)

# Invoke
messages = [HumanMessage(content="What is the crypto market sentiment?")]
messages = pre_built_agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the crypto market sentiment?
================================== Ai Message ==================================
Tool Calls:
  brave_search_tool (call_njLi788RuFtknpl2JsCxgJgN)
 Call ID: call_njLi788RuFtknpl2JsCxgJgN
  Args:
    query: crypto market sentiment October 2023
================================= Tool Message =================================
Name: brave_search_tool

Error: NotImplementedError('StructuredTool does not support sync invocation.')
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  brave_search_tool (call_3d09sNsTgWJ0tZ96WqicPnqg)
 Call ID: call_3d09sNsTgWJ0tZ96WqicPnqg
  Args:
    query: current crypto market sentiment
================================= Tool Message =================================
Name: brave_search_tool

Error: NotImplementedError('StructuredTool does not support sync invocation.')
 Please